In [3]:
.libPaths(c(.libPaths(), "/home/natasamortvanski@vhio.org/miniconda3/envs/new_env/envs/care_shiny/lib/R/library"))

library(shiny)
library(ggplot2)
library(tidyverse)
library(bslib)
library(thematic)
library(readxl)  # For reading Excel files
library(stringr)  # For string operations
library(DT)


# -------------------------------------------------------
# Updated Plot functions with risk-based coloring (FIXED)
# -------------------------------------------------------
plot_var_with_patients_gg <- function(data_train, data_pat, var,
                                      nbins = 30,
                                      title = NULL,
                                      xlab = NULL,
                                      patient_label = "Patient") {

  # 1) Checks
  if (!var %in% names(data_train))
    stop("'", var, "' not found in data_train.")
  if (!is.numeric(data_train[[var]]))
    stop("'", var, "' must be numeric.")

  if (!var %in% names(data_pat))
    stop("'", var, "' not found in uploaded patient data.")

  # 2) Extract patient values with risk info
  patient_vals <- data_pat %>%
    select(Patient_ID, value = .data[[var]], Group) %>%
    filter(!is.na(value))
  
  # Add default Group if missing
  if (!"Group" %in% colnames(patient_vals)) {
    patient_vals$Group <- "Low"
  }

  if (nrow(patient_vals) == 0)
    stop("No valid patient values for ", var)

  # 3) Default labels
  if (is.null(title)) title <- var
  if (is.null(xlab))  xlab  <- var

  # 4) Histogram stats
  h <- hist(data_train[[var]], plot = FALSE, breaks = nbins)
  y_max <- max(h$counts, na.rm = TRUE)

  # 5) Create color mapping
  color_mapping <- c("High" = "#DA4E25", "Low" = "#6BC881")
  
  # Ensure all groups have colors
  patient_vals$color <- ifelse(patient_vals$Group == "High", "#DA4E25", "#6BC881")
  
  # 5) Base histogram
  p <- ggplot(data_train, aes(x = .data[[var]])) +
    geom_histogram(
      bins = nbins,
      fill = "#8B8AC1",
      color = "white",
      alpha = 0.7
    ) +
    labs(
      title = title,
      x = xlab,
      y = "Nº de pacients"
    ) +
    theme_minimal(base_size = 14) +
    theme(
      axis.title.x = element_text(size = 18, face = "bold"),
      axis.title.y = element_text(size = 18, face = "bold"),
      axis.text.x  = element_text(size = 14),
      axis.text.y  = element_text(size = 14),
      plot.title   = element_text(size = 20, face = "bold", hjust = 0.5)
    )

  # 6) Add ONE vertical line per patient with risk-based color
  # Use manual coloring instead of scale_color_manual
  for (i in 1:nrow(patient_vals)) {
    p <- p +
      geom_vline(
        xintercept = patient_vals$value[i],
        color = patient_vals$color[i],
        linewidth = 1.5,
        alpha = 0.8
      )
  }

  # 7) Labels per patient with risk-based color
  for (i in 1:nrow(patient_vals)) {
    p <- p +
      annotate(
        "text",
        x = patient_vals$value[i],
        y = y_max * 0.5,
        label = patient_vals$Patient_ID[i],
        color = patient_vals$color[i],
        angle = 90,
        vjust = -0.5,
        hjust = 0,
        size = 5,
        fontface = "bold"
      )
  }

  p
}

# Updated categorical plot with risk-based coloring (FIXED)
plot_cat_with_patient_gg <- function(data_train, data_pat, var,
                                     title = NULL,
                                     xlab = NULL,
                                     x_labels = NULL,
                                     patient_label = "Patient",
                                     angle = 45) {
  
  # 1) Check if variable exists
  if (!var %in% names(data_train))
    stop("'", var, "' not found in data_train.")
  
  if (!var %in% names(data_pat))
    stop("'", var, "' not found in uploaded patient data.")
  
  # 2) Extract patient values with risk info
  patient_vals <- data_pat %>%
    select(Patient_ID, value = .data[[var]], Group) %>%
    mutate(value = as.character(value)) %>%
    filter(!is.na(value))
  
  # Add default Group if missing
  if (!"Group" %in% colnames(patient_vals)) {
    patient_vals$Group <- "Low"
  }
  
  if (nrow(patient_vals) == 0)
    stop("No valid patient values for ", var)
  
  # 3) Default labels
  if (is.null(title)) title <- var
  if (is.null(xlab))  xlab  <- var
  
  # 4) Counts (drop NA)
  df_counts <- data_train %>%
    mutate(.cat = as.character(.data[[var]])) %>%
    filter(!is.na(.cat)) %>%
    count(.cat, name = "n")
  
  # 5) Decide x scale limits and labels
  if (!is.null(x_labels)) {
    # Keep only categories that exist in data (prevents empty levels)
    limits_use <- intersect(names(x_labels), df_counts$.cat)
    df_counts <- df_counts %>% filter(.cat %in% limits_use)
    
    x_scale <- scale_x_discrete(
      limits = limits_use,
      labels = x_labels
    )
  } else {
    x_scale <- scale_x_discrete()
  }
  
  # 6) Create base plot - all bars same color
  p <- ggplot(df_counts, aes(x = .cat, y = n)) +
    geom_col(fill = "#8B8AC1", color = "white", width = 0.75) +
    x_scale +
    labs(
      title = title,
      x = xlab,
      y = "Nº de pacients"
    ) +
    theme_minimal(base_size = 14) +
    theme(
      axis.title.x = element_text(size = 18, face = "bold"),
      axis.title.y = element_text(size = 18, face = "bold"),
      axis.text.x  = element_text(size = 14, angle = angle, hjust = 1, vjust = 1),
      axis.text.y  = element_text(size = 14),
      plot.title   = element_text(size = 20, face = "bold", hjust = 0.5),
      panel.grid.minor = element_blank()
    )
  
  # 7) Add patient markers with risk-based coloring
  # Create color mapping
  patient_vals <- patient_vals %>%
    mutate(color = ifelse(Group == "High", "#DA4E25", "#6BC881"))
  
  for (i in 1:nrow(patient_vals)) {
    patient_id <- patient_vals$Patient_ID[i]
    patient_cat <- patient_vals$value[i]
    patient_color <- patient_vals$color[i]
    
    # Find the bar for this patient's category
    patient_bar <- df_counts %>% filter(.cat == patient_cat)
    
    if (nrow(patient_bar) > 0) {
      # Add patient marker (diamond at top of bar)
      p <- p +
        annotate(
          "point",
          x = patient_cat,
          y = patient_bar$n[1],
          color = patient_color,
          size = 4,
          shape = 18
        )
      
      # Add patient label
      if (!is.null(x_labels) && patient_cat %in% names(x_labels)) {
        label_text <- paste0(patient_id, ": ", x_labels[[patient_cat]])
      } else {
        label_text <- paste0(patient_id, ": ", patient_cat)
      }
      
      p <- p +
        annotate(
          "text",
          x = patient_cat,
          y = patient_bar$n[1] * 1.08,
          label = label_text,
          color = patient_color,
          vjust = -0.8,
          fontface = "bold",
          size = 4.5
        )
    }
  }
  
  # Adjust y-axis to accommodate labels
  y_max <- max(df_counts$n, na.rm = TRUE)
  p <- p + coord_cartesian(ylim = c(0, y_max * 1.25))
  
#   # Add simple legend for risk groups (if needed)
#   if (any(patient_vals$Group == "High") && any(patient_vals$Group == "Low")) {
#     # Create a separate dataframe for legend
#     legend_df <- data.frame(
#       x = c(0.5, 1.5),
#       y = rep(y_max * 1.19, 2),
#       label = c("Low Risk", "High Risk"),
#       color = c("#6BC881", "#DA4E25")
#     )
    
#     # Add legend as separate annotations
#     p <- p +
#       annotate("segment", x = 0.3, xend = 2.7, 
#                y = y_max * 1.15, yend = y_max * 1.15,
#                color = "gray70", linewidth = 0.5) +
#       annotate("point", x = legend_df$x[1], y = legend_df$y[1],
#                color = legend_df$color[1], size = 4, shape = 18) +
#       annotate("text", x = legend_df$x[1] + 0.1, y = legend_df$y[1],
#                label = legend_df$label[1], hjust = 0, size = 4, fontface = "bold") +
#       annotate("point", x = legend_df$x[2], y = legend_df$y[2],
#                color = legend_df$color[2], size = 4, shape = 18) +
#       annotate("text", x = legend_df$x[2] + 0.1, y = legend_df$y[2],
#                label = legend_df$label[2], hjust = 0, size = 4, fontface = "bold")
#   }
    if (any(patient_vals$Group == "High") && any(patient_vals$Group == "Low")) {

    p <- p +
        # separator line
        annotate("segment", x = Inf, xend = Inf ,y = Inf, yend = Inf, alpha = 0) +

        # LOW risk point
        annotate("point", x = Inf, y = Inf, color = "#6BC881", size = 4, shape = 18, hjust = 1.8, vjust = 1.8) +
        annotate("text",x = Inf, y = Inf,label = "Low Risk",hjust = 1.1,vjust = 1.8,size = 4,fontface = "bold") +

        # HIGH risk point
        annotate("point",x = Inf, y = Inf,color = "#DA4E25",size = 4,shape = 18,hjust = 1.8,vjust = 3.0) +
        annotate("text",x = Inf, y = Inf,label = "High Risk",hjust = 1.1,vjust = 3.0,size = 4,fontface = "bold")
    }

  
  p
}



# -------------------------------------------------------
# UI
# -------------------------------------------------------
ui <- page_navbar(
  theme = bs_theme(bootswatch = "pulse"),
  position = "static-top",

  title = tags$h1(
    "CARE - Clinical Assessment of Risk in Endometrial Cancer",
    class = "h6 text-secondary",
    style = "font-size: 40px; font-weight: bold;"
  ),

  header = tags$style(HTML("
    .navbar { display: flex; align-items: center; }
    .navbar-brand { margin-right: auto !important; }
    .navbar-nav { margin-left: auto !important; }
    .navbar-nav .nav-link { font-size: 22px !important; font-weight: bold; }

    .value-box .card-header { font-size: 25px !important; font-weight: bold; }
    .value-box .card-body { font-size: 18px !important; }

    .full-height-sidebar {
      position: fixed;
      top: 95px;
      left: 0;
      width: 400px;
      height: calc(100vh - 70px);
      padding: 20px;
      background-color: #343a40;
      overflow-y: auto;
      color: white;
    }

    .main-content {
      margin-left: 420px;
      padding: 20px;
      margin-top: 70px;
    }

    /* Scrollable table container */
    .table-container {
      max-height: 500px;
      overflow-y: auto;
      margin-top: 10px;
    }
    
    /* Target table text */
    .table-container table {
      color: #2B344D !important;
    }
    
    .table-container table td,
    .table-container table th {
      color: #2B344D !important;
    }
    
    /* Loading spinner */
    .shiny-spinner-output-container {
      text-align: center;
      padding: 20px;
    }
    
    /* Progress indicator */
    .progress-message {
      color: #6BC881;
      font-weight: bold;
      font-size: 16px;
      margin-bottom: 10px;
    }
  ")),

  # ---------------------------------------------------
  # CARE APP TAB
  # ---------------------------------------------------
  nav_panel(
    "CARE App",

    div(
      class = "full-height-sidebar",

      div(
        "CARE is a clinical prognostic modeling tool for Endometrial Cancer, focused on patients defined as a Non-Specific Molecular Profile (NMSP).",
        style = "font-size: 28px;"
      ),

      br(), br(),

      div("Upload patient file:", style = "font-size: 22px;"),
      br(),

      fileInput(
        "input_file",
        "Upload CSV, TSV, or XLSX file",
        accept = c(".csv", ".tsv", ".txt", ".xlsx", ".xls")
      ),

      br(), br(),
      uiOutput("progress_info"),
      
      br(),
    #   actionButton("run_analysis", "Run Analysis", 
    #                class = "btn-success btn-lg",
    #                style = "font-size: 18px; font-weight: bold; width: 100%;")
    uiOutput("runAnalysisUI")

    ),

    div(
      class = "main-content",

      layout_columns(
        card(
          card_header(
            "Prediction Results",
            class = "h6 text-success",
            style = "font-size: 24px; font-weight: bold;"
          ),
          div(
            class = "table-container",
            uiOutput("predictionTableUI")
          )
        ),

        col_widths = c(12)

      )
    )
  ),

  # ---------------------------------------------------
  # DATA EXPLORATION TAB
  # ---------------------------------------------------
  nav_panel(
    "Data Exploration",
    fluidRow(
      # First row: Continuous variables
      layout_columns(
        card(
          card_header("Distribution of age", class = "h6 text-success"),
          plotOutput("plotEdad")
        ),
        card(
          card_header("Tumor size distribution", class = "h6 text-success"),
          plotOutput("plotTT")
        ),
        card(
          card_header("IMC distribution", class = "h6 text-success"),
          plotOutput("plotimc")
        ),
        col_widths = c(4, 4, 4)
      ),
      
      # Second row: Categorical variables
      layout_columns(
        card(
          card_header("Histological type", class = "h6 text-success"),
          plotOutput("plotHistologia")
        ),
        card(
          card_header("Tumor stage (FIGO 2018)", class = "h6 text-success"),
          plotOutput("plotEstadio")
        ),
        card(
          card_header("Histological grade", class = "h6 text-success"),
          plotOutput("plotGrado")
        ),
        col_widths = c(4, 4, 4)
      )
    )
  )
)

# -------------------------------------------------------
# SERVER
# -------------------------------------------------------
server <- function(input, output, session) {

  # ---------------------------------------------------
  # Reactive values to store data
  # ---------------------------------------------------
  analysis_data <- reactiveValues(
    input_data = NULL,
    parsed_data = NULL,
    predictions = NULL,
    merged_data = NULL,
    status = "Waiting for file upload"
  )
  
  # ---------------------------------------------------
  # Read input file (flexible format)
  # ---------------------------------------------------
  observeEvent(input$input_file, {
    req(input$input_file)
    
    analysis_data$status <- "Reading input file..."
    
    # Get file extension
    filename <- input$input_file$name
    filepath <- input$input_file$datapath
    
    tryCatch({
      # Read based on file extension
      if (str_ends(filename, '\\.tsv$|\\.txt$')) {
        df <- read_tsv(filepath, show_col_types = FALSE, na = c("", "NA", "N/A"))
      } else if (str_ends(filename, '\\.csv$')) {
        df <- read_csv(filepath, show_col_types = FALSE, na = c("", "NA", "N/A"))
      } else if (str_ends(filename, '\\.xlsx$|\\.xls$')) {
        df <- read_excel(filepath, na = c("", "NA", "N/A"))
      } else {
        stop("Unsupported file format. Please upload CSV, TSV, or XLSX file.")
      }
      
      # Add Patient_ID if not present
      if (!"Patient_ID" %in% colnames(df)) {
        df$Patient_ID <- paste0("Patient_", 1:nrow(df))
      }
      
      # Save input data
      analysis_data$input_data <- df
      analysis_data$status <- "File uploaded successfully. Click 'Run Analysis' to process."
      
    }, error = function(e) {
      analysis_data$status <- paste("Error reading file:", e$message)
    })
  })
  
  # ---------------------------------------------------
  # Run parsing script directly
  # ---------------------------------------------------
    output$runAnalysisUI <- renderUI({
        req(analysis_data$input_data)  # only exists after successful upload
        
        actionButton(
            "run_analysis",
            "Run Analysis",
            class = "btn-success btn-lg",
            style = "font-size: 18px; font-weight: bold; width: 100%;"
        )
    })


  observeEvent(input$run_analysis, {
    req(analysis_data$input_data)
    
    analysis_data$status <- "Parsing data..."
    
    tryCatch({
      # Define args list that parse_test.R expects
      args <- list(
        df = analysis_data$input_data,
        out_filename = "/home/natasamortvanski@vhio.org/CARE/app/tmp_files/parsed_data.tsv"
      )
      
      # Source the script in a local environment
      source("/home/natasamortvanski@vhio.org/CARE/app/fix_parse.R", 
             local = TRUE)
      
      # The script will run and create the output file
      if (file.exists(args$out_filename)) {
        analysis_data$status <- "Parsing complete! Running prediction model..."
        
        # Read the parsed data
        analysis_data$parsed_data <- read_tsv(
          args$out_filename,
          show_col_types = FALSE
        )
        
        # Run prediction model
        run_prediction_model()
        
      } else {
        stop("Parsing script did not create output file")
      }
      
    }, error = function(e) {
      analysis_data$status <- paste("Parsing error:", e$message)
    })
  })
  
  # ---------------------------------------------------
  # Run prediction model
  # ---------------------------------------------------
  run_prediction_model <- function() {
    req(analysis_data$parsed_data)
    
    tryCatch({
      # Step 1: Save parsed data for prediction script
      temp_parsed <- "/home/natasamortvanski@vhio.org/CARE/app/tmp_files/parsed_data_for_pred.tsv"
      write_tsv(analysis_data$parsed_data, temp_parsed)
      
      analysis_data$status <- "Running prediction model..."
      
      # Step 2: Run prediction script
      pred_result <- system2(
        "bash",
        args = c("/home/natasamortvanski@vhio.org/CARE/app/run_model.sh"),
        stdout = TRUE,
        stderr = TRUE
      )
      
      cat("Prediction result:", pred_result, "\n")
      
      if (!file.exists("/home/natasamortvanski@vhio.org/CARE/app/tmp_files/output_predictions.tsv")) {
        stop("Prediction script failed to create output file.")
      }
      
      analysis_data$status <- "Processing results..."
      
      # Step 3: Read predictions
      predictions <- read_tsv(
        "/home/natasamortvanski@vhio.org/CARE/app/tmp_files/output_predictions.tsv",
        show_col_types = FALSE
      )
      
      # Ensure predictions have the right columns
      if (!all(c("Score", "Group") %in% colnames(predictions))) {
        cat("Available columns in predictions:", colnames(predictions), "\n")
        stop("Prediction file must contain 'score' and 'class' columns.")
      }
      
      # Step 4: Merge predictions with input data
      # Use Patient_ID column
      if ("Patient_ID" %in% colnames(analysis_data$input_data) && 
          "Patient_ID" %in% colnames(predictions)) {
        
        merged_data <- analysis_data$input_data %>%
          left_join(predictions %>% select(Patient_ID, Score, Group), 
                   by = "Patient_ID")
        
      } else {
        # If no Patient_ID, use row order
        predictions$Patient_ID_join <- paste0("Patient_", 1:nrow(predictions))
        analysis_data$input_data$Patient_ID_join <- paste0("Patient_", 1:nrow(analysis_data$input_data))
        
        merged_data <- analysis_data$input_data %>%
          left_join(predictions %>% select(Patient_ID_join, Score, Group), 
                   by = "Patient_ID_join") %>%
          select(-Patient_ID_join)
      }
      
      # Rename columns for consistency
      if ("class" %in% colnames(merged_data)) {
        merged_data <- merged_data %>% rename(Group = class)
      }
      
      analysis_data$predictions <- predictions
      analysis_data$merged_data <- merged_data
      analysis_data$status <- "Analysis complete!"
      
    }, error = function(e) {
      analysis_data$status <- paste("Error in prediction model:", e$message)
      cat("Error details:", e$message, "\n")
    })
  }
  
  # ---------------------------------------------------
  # Progress information
  # ---------------------------------------------------
  output$progress_info <- renderUI({
    div(
      class = "progress-message",
      analysis_data$status
    )
  })
  
  # ---------------------------------------------------
  # Load NMSP training data
  # ---------------------------------------------------
  nmsp_2 <- reactive({
    read_delim(
      "/home/natasamortvanski@vhio.org/CARE/app/IQ_Cancer_Endometrio_merged_NMSP.csv",
      delim = ";",
      show_col_types = FALSE
    )
  })
  
  # ---------------------------------------------------
  # Label definitions for categorical plots
  # ---------------------------------------------------
  labels_histologia <- list(
    "1" = "Hiperplàsia amb atípies",
    "2" = "Carcinoma endometrioide",
    "3" = "Carcinoma serós",
    "4" = "Carcinoma de cèl·lules clares",
    "5" = "Carcinoma indiferenciat",
    "6" = "Carcinoma mixt",
    "7" = "Carcinoma escamós",
    "8" = "Carcinosarcoma",
    "9" = "Altres"
  )
  
  labels_stage <- list(
    "1" = "Ia",
    "2" = "Ib",
    "3" = "II",
    "4" = "IIIa",
    "5" = "IIIb",
    "6" = "IIIc1",
    "7" = "IIIc2",
    "8" = "IVa",
    "9" = "IVb"
  )
  
  labels_grado <- list(
    "1" = "Grado 1",
    "2" = "Grado 2",
    "3" = "Grado 3"
  )

  # ---------------------------------------------------
  # Prediction results table
  # ---------------------------------------------------

# Replace the predictionTableUI output
output$predictionTableUI <- renderUI({
  req(analysis_data$merged_data)
  
  div(
    class = "table-container",
    DTOutput("predictionTableDT")  # Changed from tableOutput to DTOutput
  )
})

# Replace renderTable with renderDataTable
output$predictionTableDT <- renderDT({
  req(analysis_data$merged_data)
  
  df <- analysis_data$merged_data %>%
    select(Patient_ID, edad, tamano_tumoral, imc, Score, Group) %>%
    mutate(
      Score = round(Score, 3),
      Group = factor(Group, levels = c("Low", "High"))
    ) %>%
    arrange(desc(Score))
  
  datatable(
    df,
    options = list(
      pageLength = 10,
      scrollX = TRUE,
      dom = 'Bfrtip',
      buttons = c('copy', 'csv', 'excel')
    ),
    rownames = FALSE,
    class = 'display compact stripe hover'
  ) %>%
    formatStyle(
      'Group',
      backgroundColor = styleEqual(
        c('Low', 'High'),
        c('#6BC881', '#DA4E25')
      ),
      color = styleEqual(
        c('Low', 'High'),
        c('white', 'white')  # Text color
      ),
      fontWeight = 'bold'
    ) %>%
    formatStyle(
      'Score',
      background = styleColorBar(df$Score, 'lightblue'),
      backgroundSize = '100% 90%',
      backgroundRepeat = 'no-repeat',
      backgroundPosition = 'center'
    )
})


  # ---------------------------------------------------
  # Summary statistics with UI wrappers
  # ---------------------------------------------------
  output$patientCountUI <- renderUI({
    req(analysis_data$input_data)
    textOutput("patientCount")
  })
  
  output$patientCount <- renderText({
    req(analysis_data$input_data)
    paste(nrow(analysis_data$input_data), "patient(s)")
  })
  
  output$averageScoreUI <- renderUI({
    req(analysis_data$merged_data)
    textOutput("averageScore")
  })
  
  output$averageScore <- renderText({
    req(analysis_data$merged_data)
    round(mean(analysis_data$merged_data$Score, na.rm = TRUE), 3)
  })
  
  output$highRiskCountUI <- renderUI({
    req(analysis_data$merged_data)
    textOutput("highRiskCount")
  })
  
  output$highRiskCount <- renderText({
    req(analysis_data$merged_data)
    sum(analysis_data$merged_data$Group == "High", na.rm = TRUE)
  })

  # ---------------------------------------------------
  # Data exploration plots - Continuous variables
  # ---------------------------------------------------
  output$plotEdad <- renderPlot({
    req(analysis_data$merged_data, nmsp_2())
    plot_var_with_patients_gg(
      data_train = nmsp_2(),
      data_pat = analysis_data$merged_data,
      var = "edad",
      nbins = 30,
      title = "",
      xlab = "Edat"
    )
  })

  output$plotTT <- renderPlot({
    req(analysis_data$merged_data, nmsp_2())
    plot_var_with_patients_gg(
      data_train = nmsp_2(),
      data_pat = analysis_data$merged_data,
      var = "tamano_tumoral",
      nbins = 30,
      title = "",
      xlab = "Tamany tumoral"
    )
  })

  output$plotimc <- renderPlot({
    req(analysis_data$merged_data, nmsp_2())
    plot_var_with_patients_gg(
      data_train = nmsp_2(),
      data_pat = analysis_data$merged_data,
      var = "imc",
      nbins = 30,
      title = "",
      xlab = "IMC"
    )
  })

  # ---------------------------------------------------
  # Categorical plots
  # ---------------------------------------------------
  output$plotHistologia <- renderPlot({
    req(analysis_data$merged_data, nmsp_2())
    
    if ("histo_defin" %in% names(nmsp_2())) {
      plot_cat_with_patient_gg(
        data_train = nmsp_2(),
        data_pat = analysis_data$merged_data,
        var = "histo_defin",
        title = "Tipus histològic",
        xlab = "Tipus histològic",
        x_labels = labels_histologia,
        angle = 40
      )
    } else {
      ggplot() +
        annotate("text", x = 0.5, y = 0.5, 
                label = "Variable 'histo_defin' not found in training data",
                size = 6) +
        theme_void()
    }
  })
  
  output$plotEstadio <- renderPlot({
    req(analysis_data$merged_data, nmsp_2())
    
    stage_col <- NULL
    if ("estadificacion_" %in% names(nmsp_2())) {
      stage_col <- "estadificacion_"
    } else if ("estadio" %in% names(nmsp_2())) {
      stage_col <- "estadio"
    } else if ("stage" %in% names(nmsp_2())) {
      stage_col <- "stage"
    }
    
    if (!is.null(stage_col)) {
      plot_cat_with_patient_gg(
        data_train = nmsp_2(),
        data_pat = analysis_data$merged_data,
        var = stage_col,
        title = "Estadi (FIGO 2018)",
        xlab = "Estadi",
        x_labels = labels_stage,
        angle = 40
      )
    } else {
      ggplot() +
        annotate("text", x = 0.5, y = 0.5, 
                label = "Stage variable not found in training data",
                size = 6) +
        theme_void()
    }
  })
  
  output$plotGrado <- renderPlot({
    req(analysis_data$merged_data, nmsp_2())
    
    grade_col <- NULL
    if ("grado_histologico" %in% names(nmsp_2())) {
      grade_col <- "grado_histologico"
    } else if ("grado" %in% names(nmsp_2())) {
      grade_col <- "grado"
    } else if ("grade" %in% names(nmsp_2())) {
      grade_col <- "grade"
    }
    
    if (!is.null(grade_col)) {
      plot_cat_with_patient_gg(
        data_train = nmsp_2(),
        data_pat = analysis_data$merged_data,
        var = grade_col,
        title = "Grau histològic",
        xlab = "Grau",
        x_labels = labels_grado,
        angle = 0
      )
    } else {
      ggplot() +
        annotate("text", x = 0.5, y = 0.5, 
                label = "Grade variable not found in training data",
                size = 6) +
        theme_void()
    }
  })

}

shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:3486

Warning message:
“Removed 31 rows containing non-finite outside the scale range (`stat_bin()`).”
Warning message:
“Removed 8 rows containing non-finite outside the scale range (`stat_bin()`).”
Warning message in annotate("point", x = Inf, y = Inf, color = "#6BC881", size = 4, :
“Ignoring unknown parameters: `hjust` and `vjust`”
Warning message in annotate("point", x = Inf, y = Inf, color = "#DA4E25", size = 4, :
“Ignoring unknown parameters: `hjust` and `vjust`”
Warning message in annotate("point", x = Inf, y = Inf, color = "#6BC881", size = 4, :
“Ignoring unknown parameters: `hjust` and `vjust`”
Warning message in annotate("point", x = Inf, y = Inf, color = "#DA4E25", size = 4, :
“Ignoring unknown parameters: `hjust` and `vjust`”
